In [6]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

path = 'data/'

In [7]:
# import the redcap data
# data\Deidentified REDCap Survey Data 2024.08.26.csv
redcap = pd.read_csv(path + 'Deidentified REDCap Survey Data 2024.08.26.csv')
redcap['visit_date'] = pd.to_datetime(redcap['visit_date']).dt.strftime('%Y-%m-%d')
temp = redcap.copy()
temp['visit_date'] = pd.to_datetime(temp['visit_date'])
temp = temp[['encrypt_mrn', 'ed_screened', 'visit_date', 'new_diagnosis', 'pcp_followup']]
temp = temp.drop_duplicates(subset='encrypt_mrn', keep='first')

In [ ]:
# import ed diabetes data
# data\Deidentified ED Diabetes Screening File 2024.08.23.csv
ed_diabetes = pd.read_csv(path + 'Deidentified ED Diabetes Screening File 2024.08.23.csv')
def convert_to_cm(height):
    try:
        feet, inches = height.split("'")
        inches = inches.replace('"', '').strip() 
        feet = int(feet.strip()) 
        inches = float(inches) 
        return round(feet * 30.48 + inches * 2.54, 2)
    except Exception as e:
        print(f"Error converting height: {height}, {e}")
        return None
ed_diabetes['height'] = ed_diabetes['height'].apply(convert_to_cm)
ed_diabetes['height'] = ed_diabetes['height'].round(2)
ed_diabetes['weight'] = ed_diabetes['weight'].round(2)
ed_diabetes['bmi'] = ed_diabetes['bmi'][(ed_diabetes['bmi'] >= 15) & (ed_diabetes['bmi'] <= 60)]
# not english or spanish will be others
ed_diabetes['language'] = ed_diabetes['language'].apply(lambda x: x if x in ['English', 'Spanish'] else 'Other')

In [9]:
# Perform the left merge
merged = pd.merge(temp, ed_diabetes, on=['encrypt_mrn'], how='left')
merged = merged.drop_duplicates(subset='encrypt_mrn', keep='first')


In [ ]:
merged

In [11]:
merged.to_csv('processed_data/ed_data.csv', index=False)